In [1]:
!pip install lmdb

import argparse
import os
import numpy as np
import math
import itertools
import signal
import pandas as pd
import random
from sklearn.metrics import mean_absolute_error, r2_score
import lmdb
import pickle
from io import BytesIO
from sklearn.preprocessing import StandardScaler
import torchvision.transforms as transforms
from torchvision.utils import save_image
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torch.autograd import Variable
from torchvision import models

import gc

import zipfile
from io import BytesIO

import torch.nn as nn
import torch.nn.functional as F
import torch

import matplotlib.pyplot as plt
from PIL import Image

from torch.amp import GradScaler, autocast
from torch.utils.data import random_split

import glob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 19.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MDT/ProyectoIndividual/data/datos_coches_procesados.csv')

In [5]:
import zipfile
import os

ruta_zip = "/content/drive/MyDrive/Colab Notebooks/MDT/ProyectoIndividual/data/imagenes_comprimidas.zip"
carpeta_destino = "dataset_extraido"

os.makedirs(carpeta_destino, exist_ok=True)

with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
    zip_ref.extractall(carpeta_destino)

print("Descompresión completada.")

Descompresión completada.


In [6]:
# Ruta a imágenes y CSV
root_dir = "dataset_extraido/imagenes_comprimidas"
csv_path = "/content/drive/MyDrive/Colab Notebooks/MDT/ProyectoIndividual/data/datos_coches_procesados.csv"
lmdb_path = "dataset.lmdb"

# Leer CSV
df = pd.read_csv(csv_path)
df = df.set_index('url')


# Crear base de datos LMDB
env = lmdb.open(lmdb_path, map_async=True, map_size=20*1024**3, meminit=False, writemap=True, lock=False)

with env.begin(write=True) as txn:
    for url in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, url)
        if not os.path.isdir(folder_path) or url not in df.index:
            continue

        images = []
        df_row = df.loc[url]
        precio = df_row['precio']
         # Cargar todas las imágenes de la subcarpeta
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            with open(img_path, 'rb') as f:
                img_bytes = f.read()
                images.append((img_name, img_bytes))  # Guardamos el nombre y el contenido

        if not images:
            continue

        key = url.encode('utf-8')  # Solo el nombre de la carpeta como clave
        metadata = df.loc[url].to_dict()
        value = pickle.dumps({
            "images": images,    # Lista de (nombre, bytes)
            "precio": precio
        })

        txn.put(key, value)

In [7]:
random.seed(42)
def split_keys(path, train, dev):
  env = lmdb.open(path, readonly=True, lock=False)
  keys = []

  with env.begin() as txn:
      cursor = txn.cursor()
      for key, _ in cursor:
          keys.append(key)

  random.shuffle(keys)

  train_split = train
  dev_split = dev

  n = len(keys)
  train_keys = keys[:int(train_split * n)]
  dev_keys = keys[int(train_split * n):int((train_split + dev_split) * n)]
  test_keys = keys[int((train_split + dev_split) * n):]
  return train_keys, dev_keys, test_keys

In [8]:
def set_scaler(y_train):
  y_train_int = []
  for y in y_train:
    y = y.split(",")[0]
    y = y.replace('.', '')
    y = int(y)
    y_train_int.append(y)
  targets = np.array(y_train_int).reshape(-1, 1)
  scaler = StandardScaler()
  scaler.fit(targets)
  return scaler

In [9]:
def get_labels_train(df, train_keys):
  keys_str = [key.decode('utf-8') for key in train_keys]
  filtered_df = df.loc[df.index.intersection(keys_str)]
  y_train = filtered_df['precio'].tolist()
  return y_train

In [10]:
class LMDBDataset(Dataset):
    def __init__(self, lmdb_path ,keys, scaler, transform=None):
        self.env = lmdb.open(lmdb_path, readonly=True, lock=False)
        self.keys = keys
        self.transform = transform or transforms.ToTensor()
        self.normalizar = False
        self.scaler = scaler
    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        with self.env.begin() as txn:
            data = pickle.loads(txn.get(key))

        images = data["images"]
        try:
          precio = data['precio'].split(",")[0]
          precio = precio.replace('.', '')
          precio = int(precio)
        except:
          precio = -1



        precio = self.normalizar_precio(precio)

        image_list = []
        for img_name, img_bytes in images:
            img = Image.open(BytesIO(img_bytes)).convert("RGB")
            img = self.transform(img)
            image_list.append(img)

        return image_list, len(images), precio
    def normalizar_precio(self, precio):
        y = self.scaler.transform([[precio]])[0][0]
        return y

In [11]:
def custom_collate_fn(batch):
    all_images = []
    all_metadata = []
    num_images_per_group = []
    precios = []
    for images, num_images, precio in batch:
        all_images.extend(images)  # añadir todas las imágenes
        num_images_per_group.append(num_images)
        precios.append(precio)

    all_images = torch.stack(all_images, dim=0)  # [total_imágenes, 3, 256, 256]

    return all_images, num_images_per_group, precios

In [12]:
train_keys, dev_keys, test_keys = split_keys(lmdb_path, 0.85, 0.05)

In [13]:
train_precios = get_labels_train(df, train_keys)
scaler = set_scaler(train_precios)

In [14]:
train_dataset = LMDBDataset(lmdb_path, train_keys, scaler)
dev_dataset = LMDBDataset(lmdb_path, dev_keys, scaler)
test_dataset = LMDBDataset(lmdb_path, test_keys, scaler)

In [15]:
class MultiImageFlexibleVGG(nn.Module):
    def __init__(self, model, feature_dim=4096):
        super().__init__()
        self.model = model

    def forward(self, images):
        device = next(self.parameters()).device
        all_imgs = images.to(device)

        with torch.no_grad():
            feats = self.model(all_imgs)
        return feats

In [16]:
class FeatLSTM(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(FeatLSTM, self).__init__()
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True, bidirectional=True)
    self.hidden_size = hidden_size
    self.regression = nn.Linear(hidden_size * 2, 1)
  def forward(self, x, lengths):
    cut_indices = torch.cumsum(lengths, dim=0)[:-1]
    split = torch.tensor_split(x, cut_indices.tolist(), dim=0)
    out = torch.tensor([]).to(device)
    for seq in split:
      _, (h, c) = self.lstm(seq)
      h = h.flatten()
      out = torch.cat((out, torch.unsqueeze(h, 0)), dim=0)

    out = self.regression(out)
    return out

In [17]:
class MultiImageModel(nn.Module):
  def __init__(self, visual_model, input_size = 1280, hidden_size = 512):
    super(MultiImageModel, self).__init__()
    self.visual_model = MultiImageFlexibleVGG(visual_model)
    self.lstm = FeatLSTM(input_size, hidden_size)
  def forward(self, images, lengths):
    feats = self.visual_model(images)
    out = self.lstm(feats, lengths)
    return out

In [18]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.should_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True


In [19]:
class Trainer():
  def __init__(self, model, dataset, dataloader, devloader, testloader, num_epochs, learning_rate, device, save, earlystopping):
    self.model = model
    self.dataset = dataset
    self.dataloader = dataloader
    self.num_epochs = num_epochs
    self.learning_rate = learning_rate
    self.device = device
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
    self.criterion = nn.MSELoss()
    self.save = save
    self.earlystopping = earlystopping
    self.devloader = devloader
    self.testloader = testloader
    self.save_path ='/content/drive/MyDrive/Colab Notebooks/MDT/ProyectoIndividual/modelos/visual/ghost_model.pth'

  def train(self):
    for epoch in range(self.num_epochs):
      self.model.train()
      for images, num_images, y in self.dataloader:
        y = torch.tensor(y, dtype=torch.float32).to(device)
        lengths = torch.tensor(num_images).to(self.device)
        images = images.to(self.device)

        self.optimizer.zero_grad()

        output = self.model(images, lengths)
        loss = self.criterion(output.flatten(), y.flatten())
        loss.backward()
        self.optimizer.step()
        print(f"Epoch {epoch+1}/{self.num_epochs}, Loss: {loss.item()}")

      eval_loss = self.eval()

      if self.save and self.earlystopping.best_loss > eval_loss:
        torch.save(self.model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/MDT/ProyectoIndividual/modelos/Visual/ghot_model.pth')

      print(f"Epoch {epoch+1}/{self.num_epochs}, Eval Loss: {eval_loss}")
      if epoch == 8:
        self.descongelar_capas()
      self.earlystopping(eval_loss)
      if self.earlystopping.should_stop:
        print("Early stopping triggered")
        break

  def eval(self):
    self.model.eval()
    losses = []
    with torch.no_grad():
      for images, num_images, y in self.devloader:
        lengths = torch.tensor(num_images).to(self.device)
        y = torch.tensor(y, dtype=torch.float32).to(device)
        images = images.to(self.device)
        output = self.model(images, lengths)
        loss = self.criterion(output.flatten(), y.flatten())
        losses.append(loss.item())
    return sum(losses)/len(losses)
  def test(self):
    self.model.eval()
    targets = []
    preds = []
    with torch.no_grad():
      for images, num_images, y in self.testloader:
        lengths = torch.tensor(num_images).to(self.device)
        y = torch.tensor(y, dtype=torch.float32).to(device)
        images = images.to(self.device)
        output = self.model(images, lengths)
        preds.append(output)
        targets.append(y)

    y_pred = torch.cat([torch.tensor(p) for p in preds]).to('cpu').numpy().flatten()
    y_true = torch.cat([torch.tensor(t) for t in targets]).to('cpu').numpy().flatten()

    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"MAE: {mae:.4f}")
    print(f"R²:  {r2:.4f}")

    return mae, r2

  def save(self, path=None):
    if path is None:
      torch.save(self.model.state_dict(), self.save_path)
    else:
      torch.save(self.model.state_dict(), path)
  def descongelar_capas(self):
    print("Descongelando capas...")
    for name, param in self.model.visual_model.named_parameters():
      if '8.3.ghost2' in name:
        param.requires_grad = True



In [20]:
import timm
v_model = timm.create_model('ghostnet_100', pretrained=True)
v_model.classifier = nn.Identity()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

In [21]:
for name, param in v_model.named_parameters():
    if 'conv_head' in name:
        param.requires_grad = True

In [22]:
for name, param in v_model.named_parameters():
    estado = "entrenable" if param.requires_grad else "congelada"
    print(f"{name:50} → {estado}")

conv_stem.weight                                   → entrenable
bn1.weight                                         → entrenable
bn1.bias                                           → entrenable
blocks.0.0.ghost1.primary_conv.0.weight            → entrenable
blocks.0.0.ghost1.primary_conv.1.weight            → entrenable
blocks.0.0.ghost1.primary_conv.1.bias              → entrenable
blocks.0.0.ghost1.cheap_operation.0.weight         → entrenable
blocks.0.0.ghost1.cheap_operation.1.weight         → entrenable
blocks.0.0.ghost1.cheap_operation.1.bias           → entrenable
blocks.0.0.ghost2.primary_conv.0.weight            → entrenable
blocks.0.0.ghost2.primary_conv.1.weight            → entrenable
blocks.0.0.ghost2.primary_conv.1.bias              → entrenable
blocks.0.0.ghost2.cheap_operation.0.weight         → entrenable
blocks.0.0.ghost2.cheap_operation.1.weight         → entrenable
blocks.0.0.ghost2.cheap_operation.1.bias           → entrenable
blocks.1.0.ghost1.primary_conv.0.weight 

In [23]:
model = MultiImageModel(v_model)

In [24]:
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)
devloader = DataLoader(dev_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

In [25]:
earlystopping = EarlyStopping(patience = 12, min_delta=0.01)

In [26]:
trainer = Trainer(model.to(device), train_dataset, trainloader, devloader, testloader, 30, 0.001, device, True, earlystopping)

In [27]:
trainer.train()

Epoch 1/30, Loss: 0.30789875984191895
Epoch 1/30, Loss: 3.5483791828155518
Epoch 1/30, Loss: 0.20626704394817352
Epoch 1/30, Loss: 0.39444392919540405
Epoch 1/30, Loss: 0.961611270904541
Epoch 1/30, Loss: 0.37757575511932373
Epoch 1/30, Loss: 0.2291688621044159
Epoch 1/30, Loss: 0.5848636031150818
Epoch 1/30, Loss: 0.8835317492485046
Epoch 1/30, Loss: 0.9277235865592957
Epoch 1/30, Loss: 0.3567045032978058
Epoch 1/30, Loss: 3.0706799030303955
Epoch 1/30, Loss: 1.0471323728561401
Epoch 1/30, Loss: 2.1496965885162354
Epoch 1/30, Loss: 0.19746193289756775
Epoch 1/30, Loss: 0.20541024208068848
Epoch 1/30, Loss: 0.250527024269104
Epoch 1/30, Loss: 0.3263586163520813
Epoch 1/30, Loss: 0.20233452320098877
Epoch 1/30, Loss: 0.25767743587493896
Epoch 1/30, Loss: 0.18159610033035278
Epoch 1/30, Loss: 7.5466837882995605
Epoch 1/30, Loss: 0.207736074924469
Epoch 1/30, Loss: 0.11529865115880966
Epoch 1/30, Loss: 0.18435397744178772
Epoch 1/30, Loss: 0.1301831305027008
Epoch 1/30, Loss: 0.1375651061

In [31]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/MDT/ProyectoIndividual/modelos/Visual/ghot_model.pth'))

<All keys matched successfully>

In [32]:
trainer = Trainer(model.to(device), train_dataset, trainloader, devloader, testloader, 30, 0.001, device, True, earlystopping)

In [33]:
trainer.test()

MAE: 0.4237
R²:  0.1352


<ipython-input-19-eea53120342c>:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.cat([torch.tensor(p) for p in preds]).to('cpu').numpy().flatten()
<ipython-input-19-eea53120342c>:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_true = torch.cat([torch.tensor(t) for t in targets]).to('cpu').numpy().flatten()


(0.42374205589294434, 0.13522017002105713)